# Remeshing - STL, .msh files and PyGmsh
The I don't know how much'th attempt at remeshing. See word doc for the full steps overview.

*Update* 22/05/2025: It now works with .msh files. Remember that gmsh by default stores meshes with Meshio.

**Notes:** 
* while the meshes that are saved as vtu by the plasticity model do contain all the points in a cell, the `.stl` file generated below does not. "Cell" gives you a line segment instead. So keep that in mind.
* pyvista `point_id` corresponds to the array index.

In [4]:
# importing the relevant packages
import pyvista as pv
import scipy.spatial as sp
import meshio
import os
import matplotlib.pyplot as plt
import numpy as onp
import jax.numpy as jnp
import pygmsh as pg
import gmsh as gm

from units_constants import ConversionFactors as uc

**Meshing with pygmsh**
* Orientation is key - all loops should go counter-clockwise around an outward-facing normal to the cell face.
* It is possible to either assign each point, line, etc. a variable name, or store them as items in a list and call them by index
* Where points and line segments need to be ordered when generating lines and line loops, respectively, the surfaces making up a *water-tight* shell do not need to be ordered to create a volume.
* As said before, surfaces forming the volume shell should be watertight, a word that comes from the analogy that if the shell is filled with water, nothing should leak out.
* Surface elements (the triangles, in `.stl` case) should connect but not overlap.
    * This initially went wrong when generating the volume from the `.stl` triangles. I set a resolution to the points - making it different from the original resolution - and this is no problem when generating a bag of triangles which is what happens up until the `add_surface()` commands. But if I then want to merge those triangles into a volume, their sizes (= resolution) don't add up because there is an equal number of triangles as in the old mesh but the sizes are different. We get overlapping triangles, and this causes overlapping 3D cells and thus an error when calling `add_volume()`. The post that pointed me to this: [link](https://hpc.montefiore.ulg.ac.be/gmsh/gmsh/-/issues/686).

In [2]:
# mesh creation has moved to the mesh generation notebook

Step 1: Read in the file of the current mesh 
Read in the file of the mesh to be remeshed and extract the surface. We're currently using Pyvista for surface extraction.

In [3]:
# 1. Read in mesh, extract surface cells.
mesh_num = '009'
mesh_14 = f"FourteenthSesh_31-03-2025/sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps/vtk/sol_001.vtu"
mesh_BEST_17 = f"C:/Users/spaan/Thesis/BEST/sil-density-model-17_80_petsc_solver_0.00053_1.0rho_1000cells_20steps_HEX8/vtk/sol_{mesh_num}.vtu"
mesh_10cell_TET4 = f'C:/Users/spaan/Thesis/BEST/TET test/sil-density-model-rebuild_sim1_petsc_solver_20steps_80compr_7.950000e-04dens_TET4/vtk/u_{mesh_num}.vtu'
mesh_20cell_TET4 = f"C:/Users/spaan/Thesis/BEST/TET test/sil-density-model-rebuild_sim10_petsc_solver_20steps_20cells_foocompr_7.950000e-04dens_10densfactor_unifieldtype_TET4/vtk/u_{mesh_num}.vtu"


new_mesh_name = "10cell_TET4_new"

current_mesh_file = mesh_10cell_TET4

# test mesh, not relevant
box_mesh_msh = "C:/Users/spaan/Thesis/Foo/sil-density-model-16-foo_80_petsc_solver_0.00265_1.0rho_1000cells_20steps_TET10/msh/box.msh"
box_mesh_stl = "box.stl" 
box_mesh_file = box_mesh_msh


file = current_mesh_file

# read in the mesh to be remeshed and set new mesh name
meshio_mesh = meshio.read(file)
pv_mesh = pv.read(file)
if file == current_mesh_file:
    pv_mesh = pv_mesh.warp_by_vector('sol', factor = 1) # don't forget to warp it!
    # convert to stl
    new_mesh_name_stl = f"{new_mesh_name}.stl"
    new_mesh_name_msh = f"{new_mesh_name}.msh"
else:
    new_mesh_name_stl = "new_box.stl"
    new_mesh_name_msh = "new_box.msh"
surf_mesh = pv_mesh.extract_surface() # contains only the surface faces, which are indeed triangles only bc STL. It also works for the current mesh file - no internal points included.

pl = pv.Plotter(shape=(1, 1), window_size = (1200, 800), image_scale = 3)
pl.add_mesh(surf_mesh, show_edges = True)
pl.screenshot(filename = current_mesh_file[:-14]+"/unmeshed.png")
pl.show()


print(surf_mesh.n_cells)

Widget(value='<iframe src="http://localhost:55443/index.html?ui=P_0x14a25774aa0_0&reconnect=auto" class="pyvis…

1200


In [13]:
pv_slice = pv_mesh.slice(normal = [0,1,0])

pl = pv.Plotter(shape=(1, 1), window_size = (1200, 800), image_scale = 3)
pl.add_mesh(pv_slice, show_edges = True)
pl.camera_position = 'xz'
#pl.show_grid()
pl.screenshot(filename = current_mesh_file[:-14]+"/unmeshed-slice_xz.png")
pl.show()


Widget(value='<iframe src="http://localhost:55443/index.html?ui=P_0x14a4fd20b30_6&reconnect=auto" class="pyvis…

### Sorting algorithm
Step 2: Sort points and line segments
Takes the points and connectivity as locked into the mesh and sorts them into a format appropriate for use in pygmsh. Identifies, sorts and stores unique points, line segments, and the line segments making up each triangle face into dicitonaries with oriented counter-clockwise to the ouside normal. 

**To-do**
* I don't remember if this also works if the line segments are not yet oriented in the correct way. I think it does, but again, not sure. Need to check.
* it doesn't work with the data files. Why not, vtu cause?

In [5]:
# store points, line segments, line loops
n_faces = surf_mesh.n_cells # surface mesh, so cells = faces
n_points = surf_mesh.n_points
print(n_faces, "cells, ", n_points, "points")

# collect points, give them names and store in a dict
point_IDs = [point_id for point_id in onp.arange(n_points)]
point_coords = [point for point in surf_mesh.points]
point_IDs_coords = dict(zip(point_IDs, point_coords))

#print(point_IDs)
line_IDs = []
line_points = []
triangle_IDs = [f'tr{triangle_id}' for triangle_id in onp.arange(n_faces)]
triangle_lines = []
non_unique_lines = [] # stores all line points, also the reverse ones
queue = []

# The Big Sort
for i in range(n_faces): 
    triangle = surf_mesh.get_cell(i) # one triangle 
    pids = triangle.point_ids  # point ids of triangle vertices
    point_coords = triangle.points # coords of triangle vertices
    lines = [[pids[0]] + [pids[1]], [pids[1]] + [pids[2]], [pids[2]] + [pids[0]]] # lines making up the triangle, not necessarily in the right orientation


    # Ensuring orientation
    """ 
    I need to carefully go over this logic again to understand it. I think I'm not doing some things right, here, which could potentially be the cause of 
    errors later on in pygmsh.

    Specifically, look at what I'm saving here, and what I'm importing in pygmsh.
    """

    triangle_line_IDs = [] # IDs of the lines that make up the triangle
    for line in lines:
        if line not in queue and line[::-1] not in queue: # if both line [a, b] and line [b, a] are not in the queue
            line_ID = f'l{line[0]}{line[1]}' # line segment name
            line_IDs.append(line_ID) # save to unique line list
            point1 = point_IDs[line[0]] # start point ID, is an integer (index of point in the point list)
            point2 = point_IDs[line[1]] # end point ID
            line_points.append([point1, point2]) # list of start and end point of line segment, e.g. [0, 1]
            non_unique_lines.append([point1, point2]) # save to list containing all lines, also the reverse counterparts
            queue.append(line) # add line to queue, happens only if this is the first instance of the line OR its counterpart

            triangle_line_IDs.append(line_ID) # add line ID to the triangle

        elif line in queue: 
            # if line is in queue, this means we've seen it before. et the line ID to reverse, add THAT ID to the triangle list
            line_ID = f'-l{line[0]}{line[1]}'
            triangle_line_IDs.append(line_ID)
            non_unique_lines.append([point2, point1])

            # pop the line from the queue because it's now been used twice
            queue_idx = queue.index(line)
            queue.pop(queue_idx)

        else: 
            # if line[::-1] in the queue, add line ID which is ridiculous
            line_ID = f'-l{line[1]}{line[0]}'
            triangle_line_IDs.append(line_ID)
            non_unique_lines.append([point2, point1])

            # pop from queue because it's now been used twice
            queue_idx = queue.index(line[::-1])
            queue.pop(queue_idx)

    # add the three lines as an item to the triangle list
    triangle_lines.append(triangle_line_IDs)

# Confirm that indeed all line segments have been used twice, e.g., that we could in theory generate a watertight mesh
if len(queue) != 0:
    print("Not all line segments used twice! What's going on?")
    print("Lines in queue: ", queue)
else:
    print("Mesh should be watertight!")
    
line_IDs_points = dict(zip(line_IDs, line_points))
triangle_IDs_lines = dict(zip(triangle_IDs, triangle_lines))

# print(point_IDs_coords.keys())
# print(point_IDs_coords.values())
# print(line_IDs_points.keys())
# print(line_IDs_points.values())
print("Cells in queue (should be zero): ", len(queue))
print("Number of triangles:", len(triangle_IDs))
print("Number of points: ", len(line_points))
# print(triangle_IDs_lines['tr0'])


1200 cells,  602 points
Mesh should be watertight!
Cells in queue (should be zero):  0
Number of triangles: 1200
Number of points:  1800


### Pygmsh mesh creation
Step 3: Remesh
Feed the sorted points, lines and curve

In [6]:
# restating everything so that I know what it's called
n_points
n_faces 
point_IDs_coords # the point IDs + coords
line_IDs_points # the line segment IDs + start/end points
triangle_IDs_lines # the triangle IDs + line segs --> line loops
non_unique_lines # all lines


with pg.geo.Geometry() as geom:
    # add points
    points = [] 
    for p in point_IDs_coords.values():
        point = geom.add_point((p[0], p[1], p[2]))
        points.append(point)

    # add lines
    lines = []
    line_IDs_geom = []
    i = 0
    for line in line_IDs_points.values():
        line_geom = geom.add_line(points[line[0]], points[line[1]])
        lines.append(line_geom)    
        line_IDs_geom.append([key for key in line_IDs_points.keys()][i])
        i+=1
    
    # ID'ing starts at 1 in fucking pygmsh
    # print(lines[0])
    
    # add curve loops
    triangle_loops = []
    j = 0
    for triangle in triangle_IDs_lines.values():
        loop_lines = []
        
        # for line_seg in triangle:
        #     if '-' not in line_seg:
        #         line_idx = line_IDs.index(line_seg)
        #         loop_line = lines[line_idx]
        #         loop_lines.append(loop_line)

        #     elif '-' in line_seg:
        #         line_seg_new = line_seg.replace('-', '')
        #         line_idx = line_IDs.index(line_seg_new)

        #         loop_line = lines[line_idx]
        #         loop_lines.append(-loop_line)

        # chatGPT fix for the above commented out ------------------------------------------------------------
        """
        Right so I think this bit just fixes sloppy coding done in the previous cell, so that is something I should fix at some point.
        """
        triangle_point_ids = []
        for line_seg in triangle:
            # line_seg = line_ID = for example l01 or -l5253
            line_seg_clean = line_seg.replace("-", "")
            p0, p1 = line_IDs_points[line_seg_clean] # so I do need the line names
            # Check if the original segment was reversed
            # this bit sets the orientation right
            if "-" in line_seg:
                triangle_point_ids.append((p1, p0))  # reversed
            else:
                triangle_point_ids.append((p0, p1))

        # left off here w. understanding
        # Rebuild line segments in proper order: connect p0->p1, p1->p2, p2->p0
        # Get the triangle's three unique vertex IDs from the segments
        v0 = triangle_point_ids[0][0]
        v1 = triangle_point_ids[0][1]
        v2 = triangle_point_ids[1][1] if triangle_point_ids[1][0] == v1 else triangle_point_ids[1][0] # asserts something

        # Rebuild proper CCW triangle line segments
        loop_line_1 = geom.add_line(points[v0], points[v1])
        loop_line_2 = geom.add_line(points[v1], points[v2])
        loop_line_3 = geom.add_line(points[v2], points[v0])

        loop_lines = [loop_line_1, loop_line_2, loop_line_3]        
        # -------------------------------------------------------------------------


        triangle_loop = geom.add_curve_loop(loop_lines)
        triangle_loops.append(triangle_loop)
        j+=1

    # create surfaces
    triangle_surfaces = []
    for tri_loop in triangle_loops:
        triangle_surface = geom.add_surface(tri_loop)
        triangle_surfaces.append(triangle_surface)

    # Create surface loop and volume
    surface_loop = geom.add_surface_loop(triangle_surfaces) # order doesn't matter here, it does for the stuff before
    volume = geom.add_volume(surface_loop)

    # Generate 3D mesh
    mesh = geom.generate_mesh()

    # Optional: Save 
    # Debug ----------------------------------------------------------------
    # 19/05/2025: does not work with .msh file. 
    # This is the PyGmsh internal save option so I don't get why it doesn't work.
    mesh.write(new_mesh_name_msh, file_format="gmsh22") # a bug fix for saving and incidentally also the version that jax-fem uses
              
print(new_mesh_name_msh)

Warning: Appending zeros to replace the missing physical tag data.

Warning: Appending zeros to replace the missing geometrical tag data.

10cell_TET4_new.msh


In [7]:
new_mesh = new_mesh_name_msh
remeshed = pv.read(new_mesh)
#pv.plot(remeshed, show_edges=True, color='green')

pl2 = pv.Plotter(shape=(1, 1), window_size = (1200, 800), image_scale = 3)
pl2.add_mesh(remeshed, show_edges = True)
pl2.remove_scalar_bar()
pl2.screenshot(filename = current_mesh_file[:-14]+"/remeshed.png")
pl2.show()


Widget(value='<iframe src="http://localhost:55443/index.html?ui=P_0x14a4fd21430_2&reconnect=auto" class="pyvis…

In [15]:
remeshed_slice = remeshed.slice(normal = [0,1,0])
pl2 = pv.Plotter(shape=(1, 1), window_size = (1200, 800), image_scale = 3)
pl2.add_mesh(remeshed_slice, show_edges = True, color = 'Yellow')
#pl.remove_scalar_bar()
pl2.camera_position = 'xz'
#pl2.show_grid()
pl2.screenshot(filename = current_mesh_file[:-14]+"/remeshed-slice_xz.png")
pl2.show()



Widget(value='<iframe src="http://localhost:55443/index.html?ui=P_0x14a48d46e40_7&reconnect=auto" class="pyvis…

In [9]:
# # plot check whether we only have surface points, and whether they are ordered per cell
# cell0 = surf_mesh.get_cell(0)
# cell1 = surf_mesh.get_cell(1)
# cell2 = surf_mesh.get_cell(2)

# pl = pv.Plotter()
# pl.add_mesh(pv_mesh, opacity = 0.5)
# pl.add_mesh(cell0.points, color = 'red')
# pl.add_mesh(cell1.points, color = 'red')
# pl.add_mesh(cell2.points, color = 'red')
# pl.show()

# pv_mesh

### Cell aspect ratios
This is to test the general algorithm and functions. I don't need to implement this in the model code, I can put it in the analysis notebook(s). Since I have the mesh at each step, easy to analyse the aspect ratios at every compression step.

* Step 1. Algorithm for mock cell data
* Step 2. Replace mock cell with actual cell
* Step 3. Automate for all cells

Throw in big function.

In [10]:
vertices = onp.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

# 1) Get tetras and determine maximum edge length
# get tetras
# get lengths
edges = [
    onp.linalg.norm(vertices[0] - vertices[1]),
    onp.linalg.norm(vertices[0] - vertices[2]),
    onp.linalg.norm(vertices[0] - vertices[3]), 
    onp.linalg.norm(vertices[1] - vertices[2]), 
    onp.linalg.norm(vertices[1] - vertices[3]), 
    onp.linalg.norm(vertices[2] - vertices[3])
]

max_edge = onp.max(edges)

# 2) Get volume, areas of base triangles and get minimum altitude
def tetra_volume(a, b, c, d):
    return (1/6)*onp.linalg.norm(onp.dot(d-a, onp.cross(d-b, d-c)))

def tri_area(a, b, d):
    return (1/2)*onp.linalg.norm(onp.cross(b-a, c-a))


# 3) Compute aspect ratios